### TODO
        self.grouped_info = {} #need to be grouped format with {'labels':[properties]...}
        #TODO: generalize this to enable random pick of any value of any property; which means
        #need to generalize the grouped_info dict maybe to a mega dict or mega list

- add_specific_node_prop function that would by chance add a specific property as part of the condition to match for a node when calling add_node()
-     def add_node(self):
        node_label, properties_list = random.choice(list(self.grouped_names.items()))
        #TODO: have intelligent alias that can avoid overlapping node labels
- node_value = f"{property_labels} {{{properties_str}}}" 需要重新整合input dataset so that不仅node可对应property，property label也可对应property
-     def add_hop(self,hop_p=0.5):
        """
        Randomly generate hops as condition to relationship based on a customizable possibility;
        the default possibility is 0.5
        """
        hop = random.randint(0,10) #TODO: see if this is reasonable

### Cook labels and properties
- Grep csv file containing node labels, relationships, property labels, and wanted properties from Memgraph
- Create a nested dictionary with {node_labels:{sub_dictionary of properties}}, the sub_dictionary contains corresponding {property_labels: properties}

In [3]:
import re
import random
import json
import pandas as pd

# Open and load the graph schema json file
with open('/Users/yufeimeng/Desktop/KGCypherGenerator/schema.json', 'r',encoding='utf-8-sig') as file:
    schema = json.load(file)
    
# Extract nodes and edges from the schema
#labels = [node['labels'][0] for node in schema[0]['nodes']]
relationships = [relationship['type'] for relationship in schema[0]['relationships']]


# Get detailed properties from the csv file
common_names = pd.read_csv('/Users/yufeimeng/Desktop/KGCypherGenerator/memgraph-query-results-export.csv', index_col=False)

def group_labels(df, label_col, name_col):
    grouped = df.groupby(label_col)[name_col].apply(list).to_dict()
    return grouped

# Applying the function
grouped_names = group_labels(common_names, 'label', 'commonName')

#***************************************
labels = list(grouped_names.keys())
property_labels= ["commonName"] #will be generalized later

### Create alzkb nested dictionary
- In the current test case, all node labels are used; only commonName properties are selected for all nodes except that geneSymbol for Gene node is also added.

In [16]:
# type(grouped_names)
# grouped_names.keys()
# list(grouped_names.values())[0]

geneSymbol_csv = pd.read_csv('geneSymbol.csv', index_col=False)
# type(geneSymbol_csv)
# type(geneSymbol_csv['g.geneSymbol'])

geneSymbol = list(geneSymbol_csv['g.geneSymbol'])
# geneSymbol

geneSymbol_sub_dict = {}
geneSymbol_sub_dict['geneSymbol'] = geneSymbol


alzkb_nested_dict = {}
for key in grouped_names.keys():
    sub_dict = {}
    if key == 'Gene':
        sub_dict['commonName']= grouped_names[key]
        sub_dict['geneSymbol']= geneSymbol
    else:
        sub_dict['commonName']= grouped_names[key]
    alzkb_nested_dict[key] = sub_dict

# alzkb_nested_dict['Gene'].keys()



### DepthManager, Clause, Node classes

In [17]:
class DepthManager:
    _max_depth = 5  # Default maximum depth
    _min_depth = 3

    def __init__(self):
        self.depth = 0  # Starting depth

    @classmethod
    def set_max_depth(cls, depth):
        if depth > cls._min_depth:
            cls._max_depth = depth
        else:
            print("Maximum depth cannot be smaller than the min depth! \n The default max_depth is", cls._max_depth)
    
    def depth_control(self, func):
        def wrapper(*args, **kwargs):
            if self.depth == self._max_depth:
                print("Max depth reached")
                return None
            result = func(*args, **kwargs)
            self.depth += 1  # Increment depth after function call
            return result
        return wrapper
    
    def reset_depth(self):
        self.depth = 0
 
class Clause():
    def __init__(self, value, children=None):
        self.value = value
        self.children = children if children is not None else []

    def __str__(self):
        if not self.children:
            return str(self.value)
        if self.value == "RETURN":
            return f"{self.value} {', '.join(str(child) for child in self.children)}"
        return f"{self.value} {' '.join(str(child) for child in self.children)}"
    
class Node:
    """
    When called, will add connector to either nodes or relationships 
    """
    def __init__(self, value, children=None):
        self.value = value
        self.children = children if children is not None else []

    def __str__(self):
        if not self.children:
            return str(self.value)
        if self.value == '-':  
            return ' '.join(str(child) for child in self.children)
        return f"{self.value}({', '.join(str(child) for child in self.children)})"


### Query Manager Class 

In [56]:
depth_manager = DepthManager()   

class QueryManager:
    def __init__(self):
        
        self.nodes = []
        self.node_labels = []
        self.relationships = []
        
        # self.property_labels = []
        self.grouped_info = {} #need to be grouped format with {'labels':[properties]...}
        #TODO: generalize this to enable random pick of any value of any property; which means
        #need to generalize the grouped_info dict maybe to a mega dict or mega list

        self.usable_labels = set()

        self.selected_label_alias = {}

        self.parts = [] 
    
    def __str__(self):
        return "\n".join(str(node) for node in self.nodes)
    
    def import_grouped_info(self, input_group):
        if input_group:
            if type(input_group) is dict:
                self.grouped_info = input_group
                self.node_labels = list(self.grouped_info.keys())
                print("loaded node_labels:",self.node_labels)
            else:
                print("input grouped info need to be dictionary type")
        else:
            print("input grouped info cannot be empty")
    
    def import_relationships(self, input_relationships):
        if input_relationships:
            self.relationships = input_relationships
        else:
            print("relationships cannot be empty")


    def create_unique_alias(self, label):
        """Creates a unique alias for a node to prevent label overlap in queries."""
        base_alias = label.lower()
        alias = base_alias
        counter = 1
        while alias in self.usable_labels:
            alias = f"{base_alias}{counter}"
            counter += 1
        return alias
    
    def add_node(self):
        """Adds a node with random properties selected from grouped_info."""
        node_label = ''
        if self.node_labels:
            node_label = random.choice(self.node_labels) 
            # self.selected_node_label.append(node_label)
            # print(self.selected_node_label, node_label, type(self.selected_node_label))

            possible_props = self.grouped_info[node_label]
            property_label, properties_list = random.choice(list(possible_props.items()))

            alias = self.create_unique_alias(node_label)
            # self.selected_alias.append(alias)

            self.selected_label_alias[alias]=node_label

            property_value = random.choice(properties_list)
            properties_str = f"{property_label}: '{property_value}'" if possible_props else ''
            node_value = f"{node_label} {{{properties_str}}}"
            node = Node(f"({alias}:{node_value})")
            self.nodes.append(node)
            self.usable_labels.add(alias)  # Store label for possible RETURN clause usage
            return node 
        print("No node labels available. Please import grouped info first.")
        return None

        
    @depth_manager.depth_control
    def add_hop(self,hop_p=0.5):
        """
        Randomly generate hops as condition to relationship based on a customizable possibility;
        the default possibility is 0.5
        """
        current_depth = depth_manager.depth
        hop = random.randint(1,10) #TODO: see if this is reasonable
        upper_hop = hop + random.randint(1,10)
        exact_hop = f"*{hop}"
        ceiling_hop = f"*..{upper_hop}"
        floor_hop = f"*{hop}.."
        interval_hop = f"*{hop}..{upper_hop}"
        hop_choices = [exact_hop, ceiling_hop, floor_hop, interval_hop]
        if random.random() > hop_p and current_depth < depth_manager._max_depth:
            hop_choice = random.choice(hop_choices)
            return hop_choice
        else:
            return None


    @depth_manager.depth_control
    def add_relationship(self, hop_p=0.5):
        """ Randomly generate a relationship between two nodes """
        current_depth = depth_manager.depth
        rel_type = random.choice(self.relationships)
        if current_depth>=3 and random.random() > 0.5: 
            direction1 = "<-"
            direction2 = "-"
        else:
            direction1 = "-" 
            direction2 = "->"
        hop_result = self.add_hop(hop_p) if self.add_hop(hop_p) else ''
        relationship = Node(f"{direction1} [:{rel_type}{hop_result}] {direction2}")
        return Node("-", [relationship])
    

    @depth_manager.depth_control
    def add_condition(self, where_p=0.5):
        """
        Randomly generate WHERE clause based on a customizable possibility;
        the default possibility where_p is 0.5
        """
        current_depth = depth_manager.depth
        if random.random() > where_p and current_depth < depth_manager._max_depth:
            # node_label = random.choice(labels)
            # label_lower = node_label.lower()
            # property_label = random.choice(property_labels)
            # node_label = random.choice(self.selected_node_label)
            # alias = random.choice
            alias, node_label = random.choice(list(self.selected_label_alias.items()))
            print(alias, node_label)

            possible_properties = self.grouped_info[node_label]
            if possible_properties:
                property_label, properties_list = random.choice(list(possible_properties.items()))
                sample_prop_type = properties_list[0]
                value = random.randint(20, 50) if isinstance(sample_prop_type, int) else random.choice(properties_list) 
            #TODO: customize the int part

                operator = random.choice([">", "<", "=", "<=", ">="]) if isinstance(sample_prop_type, int) else '='
                return Clause("WHERE", [Clause(f"{alias}.{property_label} {operator} '{value}'", [])])
            return None
        return None

    def add_return(self, return_num=None):
        if return_num:
            random_k = random.randint(1,return_num)
            # choices = random.sample(self.usable_labels, return_num)
        usable_labels = self.get_usable_labels()
        random_k = random.randint(1,len(usable_labels))
        choices = random.sample(self.usable_labels, random_k)
        if choices:  # Check if the list is not empty
            return Clause("RETURN", choices)
        return None

    # Helper functions
    def get_usable_labels(self):
        return list(self.usable_labels)
    
    def get_parts(self):
        """ Returns a list of all nodes in a readable string format. """
        return [part for part in self.parts]
    # def get_readable_nodes(self)
    
    def reset_per_gen(self):
        """
        Only reset paras involved in each generation but doesn't reset the nested dict
        """
        self.nodes = []
        self.usable_labels = set()
        self.selected_label_alias = {}
        self.parts = []

    def reset_all(self):
        self.nodes = []
        self.node_labels = []
        self.grouped_info = {} 
        self.usable_labels = set()
        self.selected_label_alias = {}
        self.parts = []
    
        
    def is_relationship(self, part):
        """
        Determine if the given part of a query is a relationship based on containing "[]"
        Ensures that part is a string before checking.
        """
        # pattern = re.compile(r'\[(.*?)\]')
        pattern = re.compile(r'(?:-\s*\[:.*?\]\s*->|<-\s*\[:.*?\]\s*-)')
        trying = r"-\s*\[:?([A-Za-z0-9_]+)?(\*\d*(\.\.\d*)?)?\]\s*[-<>]?"
        # Ensure part is a string or bytes-like object
        if isinstance(part, str):
            # if pattern.search(part):
            if re.search(trying, part):
                return True
        return False

    
    # Integrated function
    def generate_query(self, flag=True, return_num=None, part_num=None, hop_p=0.5, where_p=0.5):
        """
        Generate cypher query with random parts; 
        part_num is by default randomly set, but can also be customized. 
        """
        def alternate_functions(flag):
            if flag:
                return self.add_node(), not flag
            else:
                return self.add_relationship(hop_p), not flag
            
        #reset paras before a new generation
        self.reset_per_gen()
        depth_manager.reset_depth()

        #flag = True  # Ensure that we start with producing nodes
        #return_num = 1 #default returns ONE random label
        if part_num is None:
            part_num = random.randint(1, depth_manager._max_depth-2)

        # Keep adding nodes and relationships while depth is within limit
        for _ in range(part_num+1):
            part, flag = alternate_functions(flag)
            if part is None:
                break
            self.parts.append(part)
        # Check if the last part is a relationship; if so, add a terminating node
        if self.parts and self.is_relationship(str(self.parts[-1])): #ensure the input part is in string format
            final_node = self.add_node()  # Generate a final node if previously ended with relationship
            if final_node:
                self.parts.append(final_node)
                print("final_node added:", final_node)

        # Optionally add a WHERE clause if depth is still under max_depth
        condition = self.add_condition(where_p)
        if condition:
            self.parts.append(condition)

        # Add RETURN clause 
        ret = self.add_return(return_num)
        if ret:
            self.parts.append(ret)

        # Create the MATCH node only if there are parts to include
        print("part_num is: ", part_num)
        return Clause("MATCH", self.parts) if self.parts else None


### Test Query Generation using Alzkb selected dict

In [39]:
# Test if alzkb one works
qm = QueryManager()
for _ in range(5):
    qm.import_grouped_info(alzkb_nested_dict)
    qm.import_relationships(relationships)
    depth_manager.set_max_depth(5)
    depth_manager.reset_depth()
    print(qm.generate_query())

loaded node_labels: ['BiologicalProcess', 'BodyPart', 'CellularComponent', 'Disease', 'Drug', 'DrugClass', 'Gene', 'MolecularFunction', 'Pathway', 'Symptom']
final_node added: (cellularcomponent:CellularComponent {commonName: 'ruffle'})
Max depth reached
part_num is:  3
MATCH (drug:Drug {commonName: '4-(5-BENZENESULFONYLAMINO-1-METHYL-1H-BENZOIMIDAZOL-2-YLMETHYL)-BENZAMIDINE'}) - [:GENEINPATHWAY*4..] -> (bodypart:BodyPart {commonName: 'carpal region'}) <- [:GENEPARTICIPATESINBIOLOGICALPROCESS] - (cellularcomponent:CellularComponent {commonName: 'ruffle'}) RETURN bodypart
loaded node_labels: ['BiologicalProcess', 'BodyPart', 'CellularComponent', 'Disease', 'Drug', 'DrugClass', 'Gene', 'MolecularFunction', 'Pathway', 'Symptom']
final_node added: (biologicalprocess:BiologicalProcess {commonName: 'regulation of Ras protein signal transduction'})
molecularfunction MolecularFunction
part_num is:  1
MATCH (molecularfunction:MolecularFunction {commonName: 'basic amino acid transmembrane transp

/var/folders/hx/mhvsgnnd2s3_cc4f_0smyv800000gn/T/ipykernel_44953/2308015174.py:147: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  choices = random.sample(self.usable_labels, random_k)


### Create Crossover/Mutation function and Fitness/Evaluation function for GP

In [60]:
# Test 
def crossover(query1, query2):
    if len(query1.nodes) > 1 and len(query2.nodes) > 1:
        # Simple crossover by swapping half of the nodes
        mid_point = min(len(query1.nodes), len(query2.nodes)) // 2
        new_nodes1 = query1.nodes[:mid_point] + query2.nodes[mid_point:]
        new_nodes2 = query2.nodes[:mid_point] + query1.nodes[mid_point:]
        query1.nodes = new_nodes1
        query2.nodes = new_nodes2

qm = QueryManager()

qm.import_grouped_info(alzkb_nested_dict)
qm.import_relationships(relationships)
depth_manager.set_max_depth(5)
depth_manager.reset_depth()
query = qm.generate_query()
query_parts = qm.get_parts()


for part in query_parts:
    print(part, type(part))
# for part in query_parts:
#     print(part)

loaded node_labels: ['BiologicalProcess', 'BodyPart', 'CellularComponent', 'Disease', 'Drug', 'DrugClass', 'Gene', 'MolecularFunction', 'Pathway', 'Symptom']
final_node added: (drug:Drug {commonName: 'Iodixanol'})
part_num is:  1
(cellularcomponent:CellularComponent {commonName: 'axon'}) <class '__main__.Node'>
- [:DISEASEASSOCIATESWITHDISEASENone] -> <class '__main__.Node'>
(drug:Drug {commonName: 'Iodixanol'}) <class '__main__.Node'>
RETURN cellularcomponent, drug <class '__main__.Clause'>


/var/folders/hx/mhvsgnnd2s3_cc4f_0smyv800000gn/T/ipykernel_44953/818492090.py:150: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  choices = random.sample(self.usable_labels, random_k)
